In [1]:
%matplotlib inline

# Train an RBM to approximate a quantum state
With RBM as a state ansatz, we need to train its parameters in order to make it close to the ground state wave function of a hamiltonian.

Notice that the cost function now is energy rather than a known dataset, we use Markov chain Monte Carlo (MCMC) method to extract the energy expectation value and gradient.

In [9]:
from __future__ import division
import numpy as np
import torch, time
import matplotlib.pyplot as plt

from rbm_torch import RBM

A model is a problem definiton, it contains state ansatz, hamiltonian, optimizer and computation graphs for wave function and gradient. Training a model using gradients requires expectation values for at least three operators, $H$, $\frac{\partial}{\partial \Theta}$ and $H\frac{\partial}{\partial \Theta}$.

In [10]:
def train(model, learning_rate, use_cuda):
    '''
    train a model.

    Args:
        model (obj): a model that meet VMC model definition.
        learning_rate (float): the learning rate for SGD.
    '''
    initial_config = np.array([-1, 1] * (model.ansatz.num_visible // 2))
    if use_cuda:
        model.ansatz.cuda()

    while True:
        # get expectation values for energy, gradient and their product,
        # as well as the precision of energy.
        energy, grad, energy_grad, precision = vmc_measure(
            model, initial_config=initial_config, num_sample=500)

        # update variables using steepest gradient descent
        g_list = [eg - energy * g for eg, g in zip(energy_grad, grad)]
        for var, g in zip(model.ansatz.parameters(), g_list):
            delta = learning_rate * g
            var.data -= delta
        yield energy, precision

The above required expectation values can be approximated by emsemble average of following operators over generated samples in VMC run, $\langle E_{\rm loc}\rangle$, $\langle\Delta_{\rm loc}\rangle$ and $\langle E_{\rm loc}\Delta_{\rm loc}\rangle$, with $E_{\rm loc}=\frac{\langle\psi|H|\sigma\rangle}{\langle\sigma|\psi\rangle}$ and $\Delta_{\rm loc}=\frac{\partial\log\langle\sigma|\psi\rangle}{\partial \Theta}$.

VMC requires a model with following property
* can propose a new configuration, given old configuration.
* can give the wave function amplitude on a spin configuration $\langle\sigma|\psi\rangle$.
* given configuration, being able to provide local quantities $E_{\rm loc}$ and $\Delta_{\rm loc}$.

As a result, VMC measurements give us desired expectation values and and error estimation for energy.

In [11]:
def vmc_measure(model, initial_config, num_bath=200, num_sample=1000, num_bin=50, measure_step=None):
    '''
    Measure an operator.

    Args:
        model (Model): model definition, requiring the following methods:
            * local_measure, get local energy and local gradient.
            * prob, get the probability of specific distribution.

        num_sample (int): number of samples.

    Return:
        number,
    '''
    if measure_step is None:
        measure_step = len(initial_config)
    print_step = num_sample * measure_step // 5

    energy_loc_list, grad_loc_list = [], []
    config = initial_config
    prob = model.prob(config)

    n_accepted = 0
    for i in range(num_bath + num_sample * measure_step):
        # generate new config and calculate probability ratio
        config_proposed = model.propose_config(config)
        prob_proposed = model.prob(config_proposed)

        # accept/reject a move by metropolis algorithm (world's most famous single line algorithm)
        if np.random.random() < prob_proposed / prob:
            config = config_proposed
            prob = prob_proposed
            n_accepted += 1

        # measurements
        if i >= num_bath and i % measure_step == 0:
            # here, I choose a lazy way that re-compute on this config, in order to get its gradients easily.
            energy_loc, grad_loc = model.local_measure(config)
            energy_loc_list.append(energy_loc)
            grad_loc_list.append(grad_loc)

        # print statistics
        if i % print_step == print_step - 1:
            print('%-10s Accept rate: %.3f' %
                  (i + 1, n_accepted * 1. / print_step))
            n_accepted = 0

    # binning statistics
    energy_loc_list = np.array(energy_loc_list)
    energy, energy_precision = binning_statistics(energy_loc_list, num_bin=num_bin)

    # get sample expectations
    grad_mean = []
    energy_grad = []
    for grad_loc in zip(*grad_loc_list):
        grad_loc = np.array(grad_loc)
        grad_mean.append(grad_loc.mean(axis=0))
        energy_grad.append(
            (energy_loc_list[(slice(None),) + (None,) * (grad_loc.ndim - 1)] * grad_loc).mean(axis=0))
    return energy, grad_mean, energy_grad, energy_precision


def binning_statistics(var_list, num_bin):
    '''
    binning statistics for variable list.
    '''
    num_sample = len(var_list)
    if num_sample % num_bin != 0:
        raise
    size_bin = num_sample // num_bin

    # mean, variance
    mean = np.mean(var_list, axis=0)
    variance = np.var(var_list, axis=0)

    # binned variance and autocorrelation time.
    variance_binned = np.var(
        [np.mean(var_list[size_bin * i:size_bin * (i + 1)]) for i in range(num_bin)])
    t_auto = 0.5 * size_bin * \
        np.abs(np.mean(variance_binned) / np.mean(variance))
    stderr = np.sqrt(variance_binned / num_bin)
    print('Binning Statistics: Energy = %.4f +- %.4f, Auto correlation Time = %.4f' %
          (mean, stderr, t_auto))
    return mean, stderr

Model definition. When proposing a new configuration, it has 5% probability to flip all spin, can making VMC sample better in Heisenberg model, proposed configuration satisfies spin conservation.


In [12]:
class VMCKernel(object):
    '''
    variational monte carlo kernel.

    Attributes:
        energy_loc (func): local energy <\sigma|H|\psi>/<\sigma|\psi>.
        ansatz (Module): torch neural network.
    '''
    def __init__(self, energy_loc, ansatz):
        self.ansatz = ansatz
        self.energy_loc = energy_loc

    def psi(self, config):
        '''
        query the wavefunction.

        Args:
            config (1darray): the bit string as a configuration.

        Returns:
            Variable: the projection of wave function on config, i.e. <config|psi>.
        '''
        psi = self.ansatz.prob_visible(torch.from_numpy(config))
        return psi

    def prob(self, config):
        '''
        probability of configuration.

        Args:
            config (1darray): the bit string as a configuration.

        Returns:
            number: probability |<config|psi>|^2.
        '''
        return abs(self.psi(config).data[0])**2

    def local_measure(self, config):
        '''
        get local quantities energy_loc, grad_loc.

        Args:
            config (1darray): the bit string as a configuration.

        Returns:
            number, list: local energy and local gradients for variables.
        '''
        psi_loc = self.psi(config)

        # get gradients {d/dW}_{loc}
        self.ansatz.zero_grad()
        psi_loc.backward()
        grad_loc = [p.grad.data/psi_loc.data[0] for p in self.ansatz.parameters()]

        # E_{loc}
        eloc = self.energy_loc(config, lambda x: self.psi(x).data, psi_loc.data)[0]
        return eloc, grad_loc

    @staticmethod
    def propose_config(old_config, prob_flip=0.05):
        '''
        flip two positions as suggested spin flips.

        Args:
            old_config (1darray): spin configuration, which is a [-1,1] string.
            prob_flip (float): the probability to flip all spins, to make VMC more statble in Heisenberg model.

        Returns:
            1darray: new spin configuration.
        '''
        # take ~ 5% probability to flip all spin, can making VMC sample better in Heisenberg model
        if np.random.random() < prob_flip:
            return -old_config

        num_spin = len(old_config)
        upmask = old_config == 1
        flips = np.random.randint(0, num_spin // 2, 2)
        iflip0 = np.where(upmask)[0][flips[0]]
        iflip1 = np.where(~upmask)[0][flips[1]]

        config = old_config.copy()
        config[iflip0] = -1
        config[iflip1] = 1
        return config

Calculate local energy

In [13]:
def heisenberg_loc(config, psi_func, psi_loc, J=1.):
    '''
    local energy for 1D Periodic Heisenberg chain.

    Args:
        config (1darray): bit string as spin configuration.
        psi_func (func): wave function.
        psi_loc (number): wave function projected on configuration <config|psi>.
        J (float): coupling strengh.

    Returns:
        number: local energy.
    '''
    # get weights and flips after applying hamiltonian \sum_i w_i|sigma_i> = H|sigma>
    nsite = len(config)
    wl, flips = [], []
    # J*SzSz terms.
    nn_par = np.roll(config, -1) * config
    wl.append(J / 4. * (nn_par).sum(axis=-1))
    flips.append(np.array([], dtype='int64'))

    # J*SxSx and J*SySy terms.
    mask = nn_par != 1
    i = np.where(mask)[0]
    j = (i + 1) % nsite
    wl += [-J / 2.] * len(i)
    flips.extend(zip(i, j))

    # calculate local energy <psi|H|sigma>/<psi|sigma>
    acc = 0
    for wi, flip in zip(wl, flips):
        config_i = config.copy()
        config_i[list(flip)] *= -1
        eng_i = wi * psi_func(config_i) / psi_loc
        acc += eng_i
    return acc

Finally, defined and run a optimization within a tensorflow session.

In [14]:
def run_demo():
    seed = 10086
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    max_iter = 200
    num_spin = 8
    num_hidden = 16
    E_exact = -3.65109341

    # visualize the loss history
    energy_list, precision_list = [], []
    def _update_curve(energy, precision):
        energy_list.append(energy)
        precision_list.append(precision)
        plt.errorbar(np.arange(1, len(energy_list) + 1), energy_list, yerr=precision_list, capsize=3)
        plt.axhline(E_exact, ls='--')
        plt.show()

    rbm = RBM(num_spin, num_hidden)
    #from random_ansatz import ModifiedBetheAnsatz
    #rbm = ModifiedBetheAnsatz(num_spin, 20, num_hidden)
    model = VMCKernel(heisenberg_loc, ansatz=rbm)

    t0 = time.time()
    for i, (energy, precision) in enumerate(train(model, learning_rate = 0.05, use_cuda = False)):
        t1 = time.time()
        print('Step %d, dE/|E| = %.4f, elapse = %.4f' % (i, -(energy - E_exact)/E_exact, t1-t0))
        _update_curve(energy, precision)
        t0 = time.time()

        # stop condition
        if i >= max_iter:
            break

In [ ]:
run_demo()